**Functions**

In [2]:
pip install linearmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 5.6 MB/s eta 0:00:00


In [ ]:
from linearmodels.iv import IV2SLS
from linearmodels.system.model import SUR
from statsmodels.multivariate.multivariate_ols import _MultivariateOLS
from multiprocessing import Pool
import numpy as np
import pandas as pd

In [ ]:
def estimate_aps(predict, X, C, S = 100, delta = 0.1, seed = 0, nprocesses = 1, chunksize = None):
    """Estimate APS for given dataset and prediction function

    Parameters
    -----------
    predict: function
        Function taking a 2D design matrix and returning a 1D vector of predictions
    X: array-like
        2D design matrix
    C: array-like
        Integer column indices for continuous variables
    S: int, default: 100
        Number of draws for each APS estimation
    delta: float, default: 0.1
        Radius of sampling ball
    seed: int, default: 0
        Seed for random number generator
    nprocesses: int, default: 1
        Number of processes used to parallelize APS estimation
    chunksize: int, default: None
        Task chunk size used to parallelize APS estimation

    Returns
    -----------
    np.ndarray
        Array of estimated APS for each observation in sample
    """

    rng = np.random.default_rng(seed)
    X = np.array(X)
    X_c = X[:, C].astype(float)
    c_std = np.std(X_c, axis=0)

    with Pool(processes=nprocesses) as pool:
        return sum(pool.starmap(estimate_aps_helper, [(i, delta, rng, X_c, c_std, X, C, predict) for i in range(S)], chunksize=chunksize))/S

In [ ]:
def estimate_aps_helper(i, delta, rng, X_c, c_std, X, C, predict):
    # Resample continuous features
    dev = rng.uniform(-delta, delta, X_c.shape)
    X_c_s = np.copy(X_c) + c_std * dev
    X_s = np.copy(X)
    X_s[:, C] = X_c_s
    return predict(X_s)

In [4]:
def estimate_treatment_effect(aps, Y, Z, D, W = None, saturated_aps = False, cov_type = "robust", weights = None):
    """Main treatment effect estimation function

    Parameters
    -----------
    aps: array-like
        Array of estimated APS values
    Y: array-like
        Array of outcome variables
    Z: array-like
        Array of treatment recommendations
    D: array-like
        Array of treatment assignments
    W: array-like, default: None
        Array of control variables
    saturated_aps: bool, default: False
        Convert APS variable into a full set of dummy variables
    cov_type: str, default: "robust"
        Covariance type of IV2SLS.
    weights: array-like, default: None
        Observation weights used in estimation

    Returns
    -----------
    tuple(IVResults, dict(D, dict(stat_label, value)))
        Tuple containing the fitted IV model and a dictionary containing the results for the treatment effect.
    """

    aps = np.array(aps)
    Y = np.array(Y)
    Z = np.array(Z)
    D = np.array(D)
    W = np.array(W)
    weights = np.array(weights)

    # Use only observations where aps is nondegenerate
    obs_tokeep = np.nonzero((aps > 0) & (aps < 1))
    print(f"We will fit on {len(obs_tokeep[0])} values out of {len(Y)} from the dataset for which the APS estimation is nondegenerate.")
    assert len(obs_tokeep[0]) > 0

    aps = aps[obs_tokeep[0]]
    Y = Y[obs_tokeep[0]]
    Z = Z[obs_tokeep[0]]
    D = D[obs_tokeep[0]]
    if W.any():
        W = W[obs_tokeep[0]]
    if weights.any():
        weights = weights[obs_tokeep[0]]

    cols = {"aps":aps, "Y":Y, "Z":Z, "D":D}
    exog = []

    # Check for single non-degeneracy
    constant = len(np.unique(aps)) == 1

    if len(W.shape) > 1:
        for i in range(W.shape[1]):
            cols["W"+str(i)] =  W[:,i]
            exog.append("W"+str(i))
            constant = (len(np.unique(W[:,i])) == 1) | constant
    elif len(W.shape) == 1:
        cols["W"] =  W
        exog.append("W")
        constant = (len(np.unique(W)) == 1) | constant

    # Add constant to specification if not provided
    if not constant:
        cols["const"] = np.ones(len(Y))
        exog.append("const")

    df = pd.DataFrame(cols)

    if saturated_aps:
        df["aps"] = df.aps.astype('category')
        dummy_df = pd.get_dummies(df.aps, prefix = "aps")
        aps_cols = list(dummy_df.columns)[1:]
        df = df.join(dummy_df[aps_cols])
        exog.extend(aps_cols)
    else:
        exog.append("aps")

    if weights.any():
        results = IV2SLS(df['Y'], df[exog], df['D'], df['Z'], weights = weights).fit(cov_type=cov_type)
    else:
        results = IV2SLS(df['Y'], df[exog], df['D'], df['Z']).fit(cov_type=cov_type)

    # Compile results
    res_dict = {"D":{}}
    res_dict["D"]['coef'] = results.params["D"]
    res_dict["D"]['stderr'] = results.std_errors["D"]
    res_dict["D"]['t'] = results.tstats["D"]
    res_dict["D"]['p'] = results.pvalues["D"]
    res_dict["D"]['n'] = results.nobs

    return results, res_dict


In [ ]:
def covariate_balance_test(aps, X, Z, W = None, saturated_aps = False, cov_type = "robust"):
    """Covariate Balance Test

    Parameters
    -----------
    aps: array-like
        Array of estimated APS values
    X: array-like
        Array of covariates to test
    Z: array-like
        Array of treatment recommendations
    W: array-like, default: None
        Array of control variables
    saturated_aps: bool, default: False
        Convert APS variable into a full set of dummy variables
    cov_type: str, default: "robust"
        Covariance type of SUR.

    Returns
    -----------
    tuple(SystemResults, dict(X, dict(stat_label, value)))
        Tuple containing the fitted SUR model and a dictionary containing results of covariate balance estimation for each covariate as well as the joint hypothesis.
    """

    aps = np.array(aps)
    X = np.array(X)
    Z = np.array(Z)
    W = np.array(W)

    # Use only observations where aps is nondegenerate
    obs_tokeep = np.nonzero((aps > 0) & (aps < 1))
    print(f"We will fit on {len(obs_tokeep[0])} values out of {len(X)} from the dataset for which the APS estimation is nondegenerate.")
    assert len(obs_tokeep[0]) > 0

    aps = aps[obs_tokeep[0]]
    X = X[obs_tokeep[0]]
    Z = Z[obs_tokeep[0]]
    if W.any():
        W = W[obs_tokeep[0]]

    cols = {"aps":aps, "Z":Z}

    dep = []
    if len(X.shape) > 1:
        for i in range(X.shape[1]):
            cols["X"+str(i)] =  X[:,i]
            dep.append("X"+str(i))
    else:
        cols["X"] =  X
        dep.append("X")

    exog = ["Z"]

    # Check for single non-degeneracy
    constant = len(np.unique(aps)) == 1

    if len(W.shape) > 1:
        for i in range(W.shape[1]):
            cols["W"+str(i)] =  W[:,i]
            exog.append("W"+str(i))
            constant = (len(np.unique(W[:,i])) == 1) | constant
    elif len(W.shape) == 1:
        cols["W"] =  W
        exog.append("W")
        constant = (len(np.unique(W)) == 1) | constant

    # Add constant to specification if not provided
    if not constant:
        cols["const"] = np.ones(len(aps))
        exog.append("const")

    df = pd.DataFrame(cols)

    if saturated_aps:
        df["aps"] = df.aps.astype('category')
        dummy_df = pd.get_dummies(df.aps, prefix = "aps")
        aps_cols = list(dummy_df.columns)[1:]
        df = df.join(dummy_df[aps_cols])
        exog.extend(aps_cols)
    else:
        exog.append("aps")

    # Covariate balance test
    mv_ols_res = SUR.multivariate_ls(df[dep], df[exog]).fit(cov_type=cov_type)

    # Joint hypothesis test: use multivariate_OLS from statsmodels
    # Edge case: single variable then joint test is the same as the original
    if len(dep) > 1:
        mv_ols_joint = _MultivariateOLS(df[dep], df[exog]).fit()
        L = np.zeros((1,len(exog)))
        L[:,0] = 1
        mv_test_res = mv_ols_joint.mv_test([("Z", L)])
    else:
        mv_test_res = None

    # Compile results
    res_dict = {}
    for x_var in dep:
        res_dict[x_var] = {}
        res_dict[x_var]['coef'] = mv_ols_res.params[f"{x_var}_Z"]
        res_dict[x_var]['stderr'] = mv_ols_res.std_errors[f"{x_var}_Z"]
        res_dict[x_var]['t'] = mv_ols_res.tstats[f"{x_var}_Z"]
        res_dict[x_var]['p'] = mv_ols_res.pvalues[f"{x_var}_Z"]
        res_dict[x_var]['n'] = int(mv_ols_res.nobs/len(dep))
    if mv_test_res:
        res_dict['joint'] = {}
        res_dict['joint']['p'] = mv_test_res.results['Z']['stat'].iloc[0, 4]
        res_dict['joint']['f'] = mv_test_res.results['Z']['stat'].iloc[0, 3]
    else:
        res_dict['joint'] = {}
        res_dict['joint']['p'] = mv_ols_res.pvalues[f"{dep[0]}_Z"]
        res_dict['joint']['t'] = mv_ols_res.tstats[f"{dep[0]}_Z"]

    return mv_ols_res, res_dict

**Data**

In [12]:
df = pd.read_stata("safety_net_elig.dta")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4633 entries, 0 to 4632
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   sum_pctg_ssi_mdcd_days  4633 non-null   float64
 1   beds                    4633 non-null   float64
 2   occupancy               4624 non-null   float64
 3   ucc_per_bed             4633 non-null   float32
 4   profit_margin           4633 non-null   float32
 5   safety_net              4633 non-null   float32
 6   safety_dollars_adj      4633 non-null   float32
 7   tot_con_sus2020_07_31   3532 non-null   float64
dtypes: float32(4), float64(4)
memory usage: 253.4 KB


In [13]:
df.head()

,sum_pctg_ssi_mdcd_days,beds,occupancy,ucc_per_bed,profit_margin,safety_net,safety_dollars_adj,tot_con_sus2020_07_31
0,0.291400,327.000000,0.800637,65056.343750,0.014022,1.0,15.635116,399.0
1,0.288200,204.000000,0.511536,64074.511719,-0.008457,1.0,9.646902,178.0
2,0.250356,326.493149,0.498117,25752.021484,0.111451,0.0,0.000000,181.0
3,0.187300,45.000000,0.278295,34279.910156,-0.048056,0.0,0.000000,53.0
4,0.223272,29.000000,0.120812,23469.007812,-0.157429,0.0,0.000000,21.0


**Assignment** Rule

In [14]:
def predict(X):
    return ((X[:,0] >= 0.202) & (X[:,1] >= 25000) & (X[:,2] <= 0.03)).astype(int)

Approximate Propensity Scores

In [15]:
# Estimate APS
df["aps"] = estimate_aps(predict = predict, X = df[["sum_pctg_ssi_mdcd_days", "ucc_per_bed", "profit_margin"]], C = [0,1,2], S = 10000, delta = 0.05, nprocesses = 1)


In [19]:
df.head()

,sum_pctg_ssi_mdcd_days,beds,occupancy,ucc_per_bed,profit_margin,safety_net,safety_dollars_adj,tot_con_sus2020_07_31,aps
0,0.291400,327.000000,0.800637,65056.343750,0.014022,1.0,15.635116,399.0,1.0000
1,0.288200,204.000000,0.511536,64074.511719,-0.008457,1.0,9.646902,178.0,1.0000
2,0.250356,326.493149,0.498117,25752.021484,0.111451,0.0,0.000000,181.0,0.0000
3,0.187300,45.000000,0.278295,34279.910156,-0.048056,0.0,0.000000,53.0,0.0000
4,0.223272,29.000000,0.120812,23469.007812,-0.157429,0.0,0.000000,21.0,0.3732


**Covariate Balance**

In [24]:
# Covariate Balance
result = covariate_balance_test(aps = df.aps, X = df[["occupancy","beds"]], Z = df.safety_net)

We will fit on 494 values out of 4633 from the dataset for which the APS estimation is nondegenerate.


In [28]:
result[0]

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                      Y   R-squared:                     -0.0613
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0666
No. Observations:                 400   F-statistic:                    0.4463
Date:                Tue, Nov 14 2023   P-value (F-stat)                0.8000
Time:                        08:07:06   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          119.14     15.895     7.4954     0.0000      87.985      150.29
aps            57.970     89.859     0.6451     0.5189     -118.15      234.09
D             -3.3768     5.0636    -0.6669     0.5048     -13.301      6.5476
==============================================================================

Endogenous: D
Instruments: Z
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x7d8234dacdc0

In [33]:
result[1]

{'D': {'coef': -3.3768356848894627,
  'stderr': 5.063594840257717,
  't': -0.6668850473663874,
  'p': 0.5048455626928656,
  'n': 400}}

**Impact Calculation**

In [30]:
# Instrumental Variables
impact = estimate_treatment_effect(aps = df.aps, Y = df.tot_con_sus2020_07_31, Z = df.safety_net, D = df.safety_dollars_adj)

We will fit on 494 values out of 4633 from the dataset for which the APS estimation is nondegenerate.


/usr/local/lib/python3.10/dist-packages/linearmodels/iv/model.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


In [31]:
impact[0]

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                      Y   R-squared:                     -0.0613
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0666
No. Observations:                 400   F-statistic:                    0.4463
Date:                Tue, Nov 14 2023   P-value (F-stat)                0.8000
Time:                        08:12:26   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          119.14     15.895     7.4954     0.0000      87.985      150.29
aps            57.970     89.859     0.6451     0.5189     -118.15      234.09
D             -3.3768     5.0636    -0.6669     0.5048     -13.301      6.5476
==============================================================================

Endogenous: D
Instruments: Z
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x7d8221ab3c70

In [32]:
impact[1]

{'D': {'coef': -3.3768356848894627,
  'stderr': 5.063594840257717,
  't': -0.6668850473663874,
  'p': 0.5048455626928656,
  'n': 400}}